In [1]:
import pandas as pd
import numpy as np

In [2]:
df_moviedetails = pd.read_csv('export_moviedetails_full_v2.csv', sep=',', encoding='iso-8859-1', escapechar='\\')

In [3]:
df_moviedetails.drop_duplicates(subset ="movie_url", keep = False, inplace = True) 

In [4]:
df_moviedetails["director"] = df_moviedetails["director"].str.replace(".","").replace(",","") 
df_moviedetails["primary_cast"] = df_moviedetails["primary_cast"].str.replace(".","").replace(",","") 
df_moviedetails["secondary_cast"] = df_moviedetails["secondary_cast"].str.replace(".","").replace(",","")
df_moviedetails["other_cast"] = df_moviedetails["other_cast"].str.replace(".","").replace(",","")

In [5]:
one_hot = df_moviedetails['genres'].str.get_dummies(sep='|')
df_moviedetails = df_moviedetails.join(one_hot)

In [6]:
pd.set_option('max_columns', None)
df_moviedetails

,movie_title,year,movie_url,plot_summary,content_rating,user_rating,director,writer,primary_cast,secondary_cast,other_cast,metascore,meta_scored,user_reviews,user_critic,plot_keywords,genres,country,language,filming_locations,budget,worldwide_gross,runtime,sound_mix,rating-enc,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,Synchronic,2019,https://www.imdb.com/title/tt9016974/,two new orleans paramedic life ripped apart e...,R,6.2,Justin Benson,Justin Benson,Anthony Mackie,Jamie Dornan,Katie Aselton,63.0,score_favorable,111.0,119.0,time travel|female time traveler|male time tra...,Drama|Horror|Sci-Fi,USA,English,"Deep South Studios, New Orleans, Louisiana, USA",0.0,1.015930e+06,102.0,na,0.0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0
1,Avengers: Endgame,2019,https://www.imdb.com/title/tt4154796/,after devastating event ofavengers infinity w...,PG-13,8.4,Anthony Russo,Christopher Markus,Robert Downey Jr,Chris Evans,Mark Ruffalo,78.0,score_favorable,9008.0,569.0,time travel|superhero|super villain|cosmic|mar...,Action|Adventure|Drama|Sci-Fi,USA,English,"Atlanta, Georgia, USA",356000000.0,2.797800e+09,181.0,Dolby Atmos,1.0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,The Friend,2019,https://www.imdb.com/title/tt9608818/,after receiving lifealtering news couple find...,R,7.2,Gabriela Cowperthwaite,Brad Ingelsby,Jason Segel,Isabella Kai,Violet McGraw,56.0,score_mixed,29.0,52.0,based on real events|singing in a car,Drama,USA,English,"Fairhope Municipal Pier and Park, Fairhope, AL...",0.0,2.937760e+05,124.0,na,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Sound of Metal,2019,https://www.imdb.com/title/tt5363618/,a heavymetal drummer life thrown freefall beg...,R,7.8,Darius Marder,Darius Marder,Riz Ahmed,Olivia Cooke,Paul Raci,81.0,score_favorable,196.0,148.0,bare chested male|deaf man|rock band|rock perf...,Drama|Music,USA,English,"Hollywood Hits Theatre, Danvers, Massachusetts...",0.0,1.351000e+04,120.0,na,1.0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
4,The Gentlemen,2019,https://www.imdb.com/title/tt8367814/,an american expat try sell highly profitable ...,R,7.8,Guy Ritchie,Guy Ritchie,Matthew McConaughey,Charlie Hunnam,Michelle Dockery,51.0,score_mixed,1405.0,278.0,british|gangster|gang|drugs|london england,Action|Comedy|Crime,UK,English,"West London Film Studios, London, England, UK",22000000.0,1.151718e+08,113.0,Dolby Digital,1.0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Beeba Boys,2015,https://www.imdb.com/title/tt4170186/,with help recent recruit gang leader take est...,na,5.0,Deepa Mehta,Deepa Mehta,Ali Kazmi,Ali Momen,Randeep Hooda,0.0,na,8.0,19.0,f rated,Crime|Drama|Thriller,Canada,English,"Vancouver, British Columbia, Canada",0.0,0.000000e+00,103.0,na,0.0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4996,Anarkali,2015,https://www.imdb.com/title/tt4947084/,the story naval officer fall love gorgeous da...,PG,6.9,Sachy,Sachy,Prithviraj Sukumaran,Biju Menon,Kabir Bedi,0.0,na,5.0,1.0,love,Drama|Musical|Romance,India,Malayalam,"Lakshadweep, India",0.0,0.000000e+00,160.0,na,0.0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0
4997,La guerre des tuques 3D,2015,https://www.imdb.com/title/tt5143700/,when winter break arrives boy friend joined n...,PG,6.1,Jean-FranÃ§ois Pouliot,Normand Canac-Marquis,Angela Galuppo,Mariloup Wolfe,Lucinda Davis,52.0,score_mixed,10.0,10.0,war and peace|tragedy|pet dog|burying a dead d...,Animation|Comedy|Drama|Family,Canada,English,na,9774485.0,9.199988e+06,82.0,na,0.0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
4998,Russell Madness,2015,https://www.imdb.com/title/tt4257950/,when man inherits grandfather arena family tr...,PG,4.2,Robert Vince,Aaron Brooks,Sean Giambrone,David Milchard,Steve Richmond,0.0,na,7.0,5.0,monkey|two word title|animal i

In [7]:
import requests
import re
from bs4 import BeautifulSoup

In [8]:
def bs_preprocess(html):
     pat = re.compile('(^[\s]+)|([\s]+$)', re.MULTILINE)
     html = re.sub(pat, '', html)       
     html = re.sub('\n', ' ', html)                                       
     html = re.sub('[\s]+<', '<', html) 
     html = re.sub('>[\s]+', '>', html) 
     return html 

In [9]:

data = {}
df_stars = pd.DataFrame(columns=["star_rank", "star_name"])

for i in range(0,10):
    r = requests.get('https://www.the-numbers.com/box-office-star-records/worldwide/lifetime-acting/top-grossing-leading-stars/'+str((100*i)+1))
    soup = BeautifulSoup(bs_preprocess(r.text), 'html.parser')
    data_items = soup.find_all("td", attrs={"class":"data"})
    for item in data_items:
        data["star_rank"] = item.string.strip()
        data["star_name"] = item.next_sibling.string.strip().replace(".","").replace(",","")
        df_stars = df_stars.append(data, ignore_index=True)

print(df_stars.head())
print("\n")
print(df_stars.tail())
print("\n")
print(df_stars.shape)

df_stars.to_csv (r'export_df_stars.csv', encoding='utf-8-sig', index = False, header=True)

df_primarycast = df_stars.rename(columns = {'star_rank': 'primary_rank', 'star_name': 'primary_cast'}, inplace = False)
df_secondarycast = df_stars.rename(columns = {'star_rank': 'secondary_rank', 'star_name': 'secondary_cast'}, inplace = False)
df_othercast = df_stars.rename(columns = {'star_rank': 'other_rank', 'star_name': 'other_cast'}, inplace = False)

  star_rank           star_name
0         1    Robert Downey Jr
1         2    Samuel L Jackson
2         3  Scarlett Johansson
3         4     Chris Hemsworth
4         5         Chris Evans


    star_rank       star_name
995       996         Gong Li
996       997    Verne Troyer
997       998  David Duchovny
998       999  Cillian Murphy
999     1,000  Kelly McGillis


(1000, 2)


In [10]:
data = {}
df_directors = pd.DataFrame(columns=["director_rank", "director"])

for i in range(0,10):
    r = requests.get('https://www.the-numbers.com/box-office-star-records/worldwide/lifetime-specific-technical-role/director/'+str((100*i)+1))
    soup = BeautifulSoup(bs_preprocess(r.text), 'html.parser')
    data_items = soup.find_all("td", attrs={"class":"data"})
    for item in data_items:
        data["director_rank"] = item.string.strip()
        data["director"] = item.next_sibling.string.strip().replace(".","").replace(",","")
        df_directors = df_directors.append(data, ignore_index=True)

print(df_directors.head())
print("\n")
print(df_directors.tail())
print("\n")
print(df_directors.shape)
df_directors.to_csv (r'export_df_directors.csv', encoding='utf-8-sig', index = False, header=True)


  director_rank          director
0             1  Steven Spielberg
1             2     Anthony Russo
2             3         Joe Russo
3             4     Peter Jackson
4             5       Michael Bay


    director_rank        director
995           996    Eddie Murphy
996           997  Fred Zinnemann
997           998   Stanley Donen
998           999  Chang-min Choo
999         1,000      Kay Cannon


(1000, 2)


In [11]:
df_moviedetails_merged = df_moviedetails.merge(df_primarycast, on='primary_cast', how='left')
df_moviedetails_merged = df_moviedetails_merged.merge(df_secondarycast, on='secondary_cast', how='left')
df_moviedetails_merged = df_moviedetails_merged.merge(df_othercast, on='other_cast', how='left')
df_moviedetails_merged = df_moviedetails_merged.merge(df_directors, on='director', how='left')

In [12]:
df_moviedetails_merged.to_csv (r'export_movie_details_merged.csv', encoding='utf-8-sig', index = False, header=True)

In [ ]:
import difflib
difflib.get_close_matches
movie_details['star_name_1'] = movie_details['primary_cast'].map(lambda x: difflib.get_close_matches(x, df_stars['star_name'], cutoff=0.5)[0] if len(difflib.get_close_matches(x, df_stars['star_name'], cutoff=0.5)) > 1 else 'no match')
movie_details